# Single-cell Analysis Workflow: 1.3M brain cells example
**Author:** [Severin Dicks](https://github.com/Intron7)

To run this notebook please make sure you have a working RAPIDS environment with all necessary dependencies. In this example workflow we'll be looking at a dataset of 1.3M brain cells from [10X Genomics](https://www.10xgenomics.com/datasets/1-3-million-brain-cells-from-e-18-mice-2-standard-1-3-0).

In [1]:
import scanpy as sc
import cupy as cp

import time
import rapids_singlecell as rsc

import warnings

warnings.filterwarnings("ignore")

In [2]:
import rmm
from rmm.allocators.cupy import rmm_cupy_allocator

rmm.reinitialize(
    managed_memory=False,  # Allows oversubscription
    pool_allocator=False,  # default is False
    devices=0,  # GPU device IDs to register. By default registers only GPU 0.
)
cp.cuda.set_allocator(rmm_cupy_allocator)

In [3]:
import gc

## Load and Prepare Data

Let's start by ensuring that we have our dataset downloaded.

In [4]:
import wget
import os
from anndata.experimental import read_elem_lazy
import h5py

url = 'https://exampledata.scverse.org/rapids-singlecell/1M_brain_cells_10X.sparse.h5ad'
data_dir = "./h5"
final = data_dir+'/nvidia_1.3M.h5ad'

if not os.path.exists(data_dir): # Check if h5 directory exists
    print('creating data directory')
    os.system('mkdir ./h5')
else:
    print(f'{data_dir} directory found')

if not os.path.isfile(final): # Check to see if we have our final output.  If it is there, get to the analysis!
    print(f'Downloading cell data into {final}...')
    wget.download(url, final)
else:
    print(f'{final} dataset found')

./h5 directory found
./h5/nvidia_1.3M.h5ad dataset found


We load the sparse count matrix from an `h5ad` file using Scanpy. The sparse count matrix will then be placed on the GPU. 

In [5]:
data_load_start = time.time()

In [ ]:
%%time
adata = sc.read(final)
adata.var_names_make_unique()
adata = adata[:1_000_000, :].copy()

We now load the the AnnData object into VRAM.

In [ ]:
%%time
rsc.get.anndata_to_GPU(adata)

Verify the shape of the resulting sparse matrix:

In [ ]:
adata.shape

In [ ]:
data_load_time = time.time()
print("Total data load and format time: %s" % (data_load_time - data_load_start))

## Preprocessing

In [ ]:
preprocess_start = time.time()

### Quality Control

We perform a basic qulitiy control and plot the results

In [ ]:
%%time
rsc.pp.flag_gene_family(adata, gene_family_name="MT", gene_family_prefix="mt-")

In [ ]:
%%time
rsc.pp.calculate_qc_metrics(adata, qc_vars=["MT"])

In [ ]:
%%time
sc.pl.scatter(adata, "total_counts", "pct_counts_MT")
sc.pl.scatter(adata, "total_counts", "n_genes_by_counts")

In [ ]:
%%time
sc.pl.violin(adata, keys="n_genes_by_counts")
sc.pl.violin(adata, keys="total_counts")
sc.pl.violin(adata, keys="pct_counts_MT")

### Filter

We filter the count matrix to remove cells with an extreme number of genes expressed.
We also filter out cells with a mitchondrial countent of more than 20%.

In [ ]:
%%time
adata = adata[
    (adata.obs["n_genes_by_counts"] < 5000)
    & (adata.obs["n_genes_by_counts"] > 500)
    & (adata.obs["pct_counts_MT"] < 20)
].copy()

Many python objects are not deallocated until garbage collection runs. When working with data that barely fits in memory (generally, >50%) you may need to manually trigger garbage collection to reclaim memory.

In [ ]:
%%time
gc.collect()

We also filter out genes that are expressed in less than 3 cells.

In [ ]:
%%time
rsc.pp.filter_genes(adata, min_cells=3)

We store the raw expression counts in the `.layer["counts"]`

In [ ]:
adata.layers["counts"] = adata.X.copy()

In [ ]:
adata.shape

### Normalize

We normalize the count matrix so that the total counts in each cell sum to 1e4.

In [ ]:
%%time
rsc.pp.normalize_total(adata, target_sum=1e4)

Next, we log transform the count matrix.

In [ ]:
%%time
rsc.pp.log1p(adata)

### Select Most Variable Genes

Now we search for highly variable genes. This function only supports the flavors `cell_ranger` `seurat` `seurat_v3` and `pearson_residuals`. As you can in scanpy you can filter based on cutoffs or select the top n cells. You can also use a `batch_key` to reduce batcheffects.

In this example we use `seurat_v3` for selecting highly variable genes based on the raw counts in `.layer["counts"]`. 

In [ ]:
%%time
rsc.pp.highly_variable_genes(
    adata, n_top_genes=5000, flavor="seurat_v3", layer="counts"
)

In [ ]:
%%time
rsc.get.anndata_to_CPU(adata, layer="counts")

Now we safe this version of the AnnData as adata.raw.

In [ ]:
%%time
adata.raw = adata

Now we restrict our AnnData object to the highly variable genes.

In [ ]:
%%time
adata = adata[:, adata.var["highly_variable"]]

In [ ]:
adata.shape

Next we regress out effects of counts per cell and the mitochondrial content of the cells. As you can with scanpy you can use every numerical column in `.obs` for this.

In [ ]:
%%time
rsc.pp.regress_out(adata, keys=["total_counts", "pct_counts_MT"])

### Scale

Finally, we scale the count matrix to obtain a z-score and apply a cutoff value of 10 standard deviations.

In [ ]:
%%time
rsc.pp.scale(adata, max_value=10)

### Principal component analysis

We use PCA to reduce the dimensionality of the matrix to its top 100 principal components. We use the PCA implementation from cuml to run this. With `use_highly_variable = False` we save VRAM since we already subset the matrix to only HVGs.

In [ ]:
%%time
rsc.pp.pca(adata, n_comps=100, use_highly_variable=False)

We can use scanpy `pca_variance_ratio` plot to inspect the contribution of single PCs to the total variance in the data.

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True, n_pcs=100)

Now we move `.X` and `.layers` out of the GPU.

In [ ]:
%%time
rsc.get.anndata_to_CPU(adata)

In [ ]:
preprocess_time = time.time()
print("Total Preprocessing time: %s" % (preprocess_time - preprocess_start))

Visualization## Clustering and Visualization

### Computing the neighborhood graph and UMAP

Next we compute the neighborhood graph using rsc.

Scanpy CPU implementation of nearest neighbor uses an approximation, while the GPU version calculates the exact graph. Both methods are valid, but you might see differences.

In [ ]:
%%time
rsc.pp.neighbors(adata, n_neighbors=15, n_pcs=50)

Next we calculate the UMAP embedding using rapdis.

In [ ]:
%%time
rsc.tl.umap(adata, min_dist=0.3)

### Clustering

Next, we use the Louvain and Leiden algorithm for graph-based clustering.

In [ ]:
%%time
rsc.tl.louvain(adata, resolution=0.6)

In [ ]:
%%time
rsc.tl.leiden(adata, resolution=1.0)

In [ ]:
%%time
sc.pl.umap(adata, color=["louvain", "leiden"], legend_loc="on data")

## TSNE

In [ ]:
%%time
rsc.tl.tsne(adata, n_pcs=40)

In [ ]:
sc.pl.tsne(adata, color=["louvain", "leiden"], legend_loc="on data")

## Differential expression analysis

We now use logistic regression to compute a ranking for highly differential genes in each Louvaincluster.

In [ ]:
%%time
rsc.tl.rank_genes_groups_logreg(adata, groupby="louvain", use_raw=False)

In [ ]:
%%time
sc.pl.rank_genes_groups(adata, n_genes=20)

## Diffusion Maps

In [ ]:
%%time
rsc.tl.diffmap(adata)
adata.obsm["X_diffmap"] = adata.obsm["X_diffmap"][:, 1:]

In [ ]:
sc.pl.diffmap(adata, color="louvain")

After this you can use `X_diffmap` for `sc.pp.neighbors` and other functions. 

In [ ]:
print("Total Processing time: %s" % (time.time() - preprocess_start))

In [ ]:
# These notebooks are very GPU memory intensive!
# In order to free up GPU memory, we'll kill this kernel prior to proceeding.  You will get a message.  This is expected.
# If you have a CUDA or an Out Of Memory (OOM) error, please kill all kernels to free up your GPU memory and try again!
# You can comment this out if you want to continue exploring the notebook.
# Please consult the README for more tips and tricks.

import IPython

IPython.Application.instance().kernel.do_shutdown(True)